In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
from collections import Counter,defaultdict
import matplotlib.image as mpimg
from IPython.display import clear_output
import time
from  sklearn.model_selection import train_test_split
import sys,os
from shutil import copyfile
#os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
## PARAMETERS

TEST_SIZE = 0.99
IMAGE_DIM = 224

In [3]:
category_img_path = '/home/rakesh/Desktop/CF Dataset/Category and Attribute Prediction/Anno/list_category_img.txt'
category_cloth_path = '/home/rakesh/Desktop/CF Dataset/Category and Attribute Prediction/Anno/list_category_cloth.txt'
img_path = '/home/rakesh/Desktop/CF Dataset/Category and Attribute Prediction/'

In [4]:
category_img = pd.read_csv(category_img_path,delim_whitespace=True)
category_cloth = pd.read_csv(category_cloth_path,delim_whitespace=True)

## CATEGORY

In [5]:
category_cloth.head()

,category_name,category_type
0,Anorak,1
1,Blazer,1
2,Blouse,1
3,Bomber,1
4,Button-Down,1


In [6]:
## root class distribution

## convert 50 class to their names
order_map = {}
for i in category_cloth.index:
    order_map[i+1] = category_cloth.iloc[i]['category_name']

    
#   distribution
category_cloth_map = defaultdict()
    
for i in category_cloth.index:
    category_cloth_map[i+1] = category_cloth.loc[i]['category_type']
    
class50_map = {}
l = []
for i in category_img['category_label']:
    l.append(category_cloth_map[i])
    
print "Root Class distrubution : "
print Counter(l)

print "\n 50 class distribution : "
dist = Counter(category_img['category_label'])
print dist

##  remove order map to convert to numerical classes
## filter minority classes
category_img.head()

Root Class distrubution : 
Counter({1: 139709, 3: 90550, 2: 58963})

 50 class distribution : 
Counter({41: 72158, 18: 36887, 3: 24557, 32: 19666, 17: 15429, 33: 14773, 6: 13311, 16: 13123, 11: 10467, 19: 10078, 2: 7495, 48: 7408, 26: 7076, 42: 6153, 30: 5013, 29: 4416, 10: 4048, 34: 3048, 44: 2294, 39: 2120, 24: 1669, 35: 1106, 15: 791, 12: 748, 9: 716, 13: 676, 27: 594, 22: 527, 23: 486, 36: 386, 5: 330, 7: 324, 4: 309, 1: 160, 47: 150, 20: 146, 43: 126, 14: 97, 21: 77, 46: 70, 37: 54, 25: 49, 28: 45, 31: 32, 8: 17, 40: 17})


,image_name,category_label
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,3
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,3
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,3
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,3
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,3


In [7]:
## --------------  VISUALIZATION -------------------

for i in class50_map.keys():
    clear_output( wait = True )
    
    img = mpimg.imread(img_path+i)
    plt.imshow(img)
    plt.show()
    print "Class : "+str(class50_map[i])
    time.sleep(1)

## DATASET PREPARATION : FILTERING MINORITY CLASSES

In [8]:
## ===========      CREATE A DATAFRAME OF IMAGE PATH AND ITS LABELS   =====================
image_path = []
image_label = []
for i in category_img.index:
    if dist[category_img.iloc[i]['category_label']]>=10070:
        image_path.append(category_img.iloc[i]['image_name'])
        image_label.append(order_map[category_img.iloc[i]['category_label']])
        class50_map[ category_img.iloc[i]['image_name']] = order_map[category_img.iloc[i]['category_label']]  

In [9]:
df1 = pd.DataFrame({'path':image_path,'label':image_label})
#df1.to_csv('dataset.csv')

In [10]:
print "DATASET CLASS DISTRIBUTION\n"
print "n_classes : "+str(len(Counter(df1['label'])))
print Counter(df1['label'])

DATASET CLASS DISTRIBUTION

n_classes : 10
Counter({'Dress': 72158, 'Tee': 36887, 'Blouse': 24557, 'Shorts': 19666, 'Tank': 15429, 'Skirt': 14773, 'Cardigan': 13311, 'Sweater': 13123, 'Jacket': 10467, 'Top': 10078})


In [11]:
## balance classes
UPPER_LIMIT = 6800

count_dict = defaultdict(lambda : 0)

labels =[]
paths =[]

for i,l in enumerate(image_label):
    if count_dict[l]<UPPER_LIMIT:
        labels.append(image_label[i])
        paths.append(image_path[i])
    count_dict[l]+=1
    
df = pd.DataFrame({'path':paths,'label':labels})
Counter(df['label'])

Counter({'Blouse': 6800,
         'Cardigan': 6800,
         'Dress': 6800,
         'Jacket': 6800,
         'Shorts': 6800,
         'Skirt': 6800,
         'Sweater': 6800,
         'Tank': 6800,
         'Tee': 6800,
         'Top': 6800})

In [12]:
## PREPARE   TRAIN , TEST , VALIDATION :STRATIFIED

X_train, X_test, y_train, y_test = train_test_split(df['path'], df['label'],stratify=df['label'], test_size=TEST_SIZE)

train = pd.DataFrame({'path':X_train,'label':y_train})
test_data = pd.DataFrame({'path':X_test,'label':y_test})

# clear variables

del X_train, X_test, y_train, y_test

print "TEST and VALIDATION data size  "
print test_data.shape

X_val, X_test, y_val, y_test = train_test_split(test_data['path'], test_data['label'],stratify=test_data['label'], test_size=0.25)

val = pd.DataFrame({'path':X_val,'label':y_val})
test = pd.DataFrame({'path':X_test,'label':y_test})

# clear variables

del X_val, X_test, y_val, y_test,df

Counter(test['label'])


#  ====== DROP INDEXES ============


val = val.reset_index(drop=True)
test = test.reset_index(drop=True)

TEST and VALIDATION data size  
(67320, 2)


In [13]:
print sum(Counter(test['label']).values())
Counter(val['label'])

16830


Counter({'Blouse': 5049,
         'Cardigan': 5049,
         'Dress': 5049,
         'Jacket': 5049,
         'Shorts': 5049,
         'Skirt': 5049,
         'Sweater': 5049,
         'Tank': 5049,
         'Tee': 5049,
         'Top': 5049})

In [15]:
## create directories and save data
dest_dir = '/home/rakesh/Desktop/CF Dataset/mod/test/'
def create_folders(data):
    
    print "Total Images : "+str(data.shape[0])
    global dest_dir
    for i in data.index:
        label = data.iloc[i]['label']
        src = img_path+data.iloc[i]['path']
        
        if not os.path.exists(dest_dir+label):
            os.makedirs(dest_dir+label)
        k = data.iloc[i]['path'].split('/')  
        k = k[-2]+'_'+k[-1]
        copyfile(src, dest_dir+label+'/'+k)
        if i%1000==0:
            print i
    
create_folders(test)    

Total Images : 16830
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000


In [ ]:
val.shape

In [ ]:
test.shape

##  VGG PRETRAINED

In [ ]:
import glob
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
from IPython.display import clear_output
import time
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.densenet import DenseNet121
import sys
from PIL import Image
import os
from keras.preprocessing.image import load_img
import numpy as np
from keras.preprocessing.image import img_to_array
import pydot
from keras_preprocessing.image import ImageDataGenerator
from keras import layers,models
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense,GlobalAveragePooling2D
from keras.models import Model
from keras import optimizers
#from classification_models import ResNet18

In [ ]:
IMAGE_BATCH = 128
N_CLASSES = 20

In [ ]:
model_vgg = VGG16(weights='imagenet',include_top=False, input_shape=(IMAGE_DIM, IMAGE_DIM, 3))

In [ ]:
model_vgg.summary()

In [ ]:
def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    total_memory = 4.0*batch_size*(shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes

## Transfer Learning

## --> DATA GENERATORS

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)
datagen_val = ImageDataGenerator(rescale=1./255)


##  for training 
generator = datagen.flow_from_directory(
                                        directory ='/home/rakesh/Desktop/CF Dataset/mod/val' ,
                                        interpolation = "nearest",
                                        target_size=(IMAGE_DIM, IMAGE_DIM),
                                        batch_size = IMAGE_BATCH,
                                        #save_to_dir = 'val_augmented/'
                                       )

##   for validation

val_generator = datagen_test.flow_from_directory(
                                        directory ='/home/rakesh/Desktop/CF Dataset/mod/test' ,
                                        interpolation = "nearest",
                                        target_size=(IMAGE_DIM, IMAGE_DIM),
                                        batch_size = IMAGE_BATCH
                                        #save_to_dir = 'val_augmented/'
                                       )


#train_X = np.zeros(shape=(val.shape[0], 7, 7, 512))
#train_y = np.zeros(shape=(val.shape[0],20))

In [ ]:
  for i,layer in enumerate(model_vgg.layers):
    print(i,layer.name,layer.trainable)

In [ ]:
x = model_vgg.output
x = GlobalAveragePooling2D()(x)

x = Dense(128, activation='relu')(x)
#x = Dense(1128, activation='relu')(x)
#x = Dense(512, activation='relu')(x)
predictions = Dense(20, activation='softmax')(x)
model = Model(inputs=model_vgg.input, outputs=predictions)

model.summary()

In [ ]:
##  freeze layers of vgg
for layer in model_vgg.layers:
    layer.trainable = False

In [ ]:
model.compile(loss='categorical_crossentropy', 
                  optimizer=optimizers.SGD(lr=0.001, momentum=0.9), 
                  metrics=['accuracy'])

In [ ]:
for i,layer in enumerate(model.layers):
    print(i,layer.name,layer.trainable)
print "Memory : " + str(get_model_memory_usage(IMAGE_BATCH,model))+" GB\n"

In [ ]:
## Train on FC layers
num_training_samples = val.shape[0]
# history = model.fit_generator(
#             generator,
#             steps_per_epoch=(num_training_samples //IMAGE_BATCH ),
#     verbose=1,
#             epochs=5)

history = model.fit_generator(
          generator,
          verbose=1,
          steps_per_epoch= (num_training_samples //IMAGE_BATCH),
          epochs=10,
          validation_data=val_generator,
        validation_steps=(num_training_samples //IMAGE_BATCH))

In [ ]:
# ## 
# for layer in model.layers[:18]:
#     layer.trainable = False

In [ ]:
import matplotlib.pyplot as plt
def show_history(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'val_accuracy'], loc='best')
    plt.show()

show_history(history)

## ==========================   TRY  ===============================

In [ ]:
##    FC LAYERS

# model = models.Sequential()
# model.add(layers.Dense(256, activation='relu', input_dim=7 * 7 * 512))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(N_CLASSES, activation='softmax'))

In [ ]:
# i = 0
# for inputs_batch, labels_batch in generator:
#     features_batch = model_vgg.predict(inputs_batch)
#     train_X[i * IMAGE_BATCH : (i + 1) * IMAGE_BATCH] = features_batch
#     train_y[i * IMAGE_BATCH : (i + 1) * IMAGE_BATCH] = labels_batch
#     i += 1
#     if i * batch_size >= test.shape[0]:
#         break
        

In [ ]:
## Small COnvnet

##features_train = model_vgg.predict_generator(generator)


# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=(3, IMAGE_DIM, IMAGE_DIM)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1))
# model.add(Activation('softmax'))

# model.compile(loss='crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])